In [1]:
%load_ext autoreload
%autoreload 2

In [121]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from bib.cleaning import *
from bib.plot import *
import math

In [111]:
contatos01 = pd.read_excel('./dados/last.xlsx',sheet_name='Contatos_01')
contatos02 = pd.read_excel('./dados/last.xlsx',sheet_name='Contatos_02')
data = pd.read_excel('./dados/last.xlsx',sheet_name='Pessoas')
Nmortos = generate_mortalidade()
Faixa = [
    [0,20],
    [20,30],
    [30,50],
    [50,70],
    [70,10000]
]
tempo = [2.5, 7.5,22.5,120,240]

In [112]:
def transform_faixa(i,Faixa):
    for j in range(len(Faixa)):
        if(i < Faixa[j][0]):
            return j - 1
    return j

In [113]:
Idade = data['Idade'].values
Idade = np.array([transform_faixa(i,Faixa) for i in Idade])
data['Faixa'] = Idade

Idade = contatos01['idade'].values
Idade = np.array([transform_faixa(i,Faixa) for i in Idade])
contatos01['Faixa'] = Idade

Idade = contatos02['idade'].values
Idade = np.array([transform_faixa(i,Faixa) for i in Idade])
contatos02['Faixa'] = Idade

,id,idade,Frequência,Pele,Duração,Faixa
0,0,68.0,1.0,2.0,5.0,3
1,0,55.0,2.0,1.0,3.0,3
2,1,29.0,1.0,1.0,5.0,1
3,1,33.0,1.0,1.0,4.0,2
4,1,3.0,1.0,1.0,5.0,0
...,...,...,...,...,...,...
19839,2032,47.0,5.0,2.0,2.0,2
19840,2032,47.0,5.0,2.0,2.0,2
19841,2032,52.0,5.0,2.0,2.0,3
19842,2032,62.0,5.0,2.0,2.0,3


In [105]:
def generate_gij(data,contatos01):
    id_ = data['id'].values
    faixas = data['Faixa'].values
    g_ij = np.zeros((5,5))
    for i,faixa in zip(id_,faixas):
        for j in contatos01[contatos01['id'] == i]['Faixa'].values:
            #print(i,j)
            g_ij[faixa][j] += 1
            g_ij[j][faixa] += 1
    return g_ij

In [106]:
g_ij01 = generate_gij(data,contatos01)
g_ij02 = generate_gij(data,contatos02)
print(g_ij01/p_i)
print(g_ij02/p_i)

In [129]:
def generate_tij(data,contatos01,tempo):
    id_ = data['id'].values
    faixas = data['Faixa'].values
    t_ij = np.zeros((5,5))
    sigma_ij = np.zeros((5,5))
    for i,faixa in zip(id_,faixas):
        df = contatos01[contatos01['id'] == i]
        for j,t in zip(df['Faixa'].values,df['Duração'].values):
            #print(i,j)
            if(not math.isnan(t)):
                t_ij[faixa][j] += tempo[int(t)-1]
                t_ij[j][faixa] += tempo[int(t)-1]
                sigma_ij[faixa][j] += tempo[int(t)-1]**2
                sigma_ij[j][faixa] += tempo[int(t)-1]**2
    return t_ij,sigma_ij

In [140]:
t_ij,sigma_ij = generate_tij(data,contatos01,tempo)

In [141]:
t_ij/p_i,sigma_ij

(array([[1122.62237762,  500.0443787 , 1469.15584416,  231.17941712,
          121.68141593],
        [  98.49358974,  775.35502959,  231.51515152,   74.90892532,
           45.21017699],
        [ 395.54195804,  316.44970414,  500.23809524,  167.21311475,
           87.95353982],
        [ 147.92249417,  243.34319527,  397.4025974 ,  455.29143898,
          248.3960177 ],
        [  32.05128205,   60.45857988,   86.04978355,  102.25409836,
          305.28761062]]),
 array([[1.89702225e+08, 1.58491812e+07, 6.83958750e+07, 2.36347938e+07,
         5.06067500e+06],
        [1.58491812e+07, 2.49464125e+07, 9.39548750e+06, 6.77470000e+06,
         1.87204375e+06],
        [6.83958750e+07, 9.39548750e+06, 2.07414625e+07, 1.39932375e+07,
         2.76004375e+06],
        [2.36347938e+07, 6.77470000e+06, 1.39932375e+07, 4.48514875e+07,
         9.34891875e+06],
        [5.06067500e+06, 1.87204375e+06, 2.76004375e+06, 9.34891875e+06,
         1.33442125e+07]]))

In [69]:
Idade = data['Idade'].values
Idade = np.array([transform_faixa(i,Faixa) for i in Idade])
data['Faixa'] = Idade
_, p_i = np.unique(Idade,return_counts = True)

In [35]:
p_i = p_i/np.sum(p_i)

In [70]:
p_i

array([858, 169, 231, 549, 226])

In [100]:
922/858

1.0745920745920745